# Interacting with LLMs from various providers using Haystack 

### Prompt node initialization - no base instructions

In [19]:
import os
from dotenv import load_dotenv
import openai
load_dotenv("./../../.env")

# Open AI API Key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
from haystack.nodes import PromptTemplate, AnswerParser, PromptNode

In [21]:
# Create a prompt node
openai_api_key = os.getenv("OPENAI_API_KEY")

pn = PromptNode("gpt-3.5-turbo", 
                api_key=openai_api_key, 
                model_kwargs={"stream":False}, 
                max_length=1000)

# Create a prompt
prompt = "What are the three most interesting things about Berlin? Be brief in your answer."


In [ ]:
# using a prompt template
tourist_recommendation_prompt = PromptTemplate(
    prompt="""You are a helpful assistant to a tourist looking find attractions in a city. \ 
        If the request is vague, ask the tourist to provide the name of the city and what kind \
        of attractions they are looking for. \ 
        You answer:""",
    output_parser=AnswerParser(),
)
pn = PromptNode("gpt-3.5-turbo", 
                api_key=openai_api_key, 
                model_kwargs={"stream":False},
                default_prompt_template=tourist_recommendation_prompt)

In [ ]:
pn.run("I am looking for a place with tropical wether in America")

Prompting LLMs from HF

In [ ]:
from haystack.nodes import PromptNode

# Initalize the node passing the model:
prompt_node_gt5 = PromptNode(model_name_or_path="google/flan-t5-xl",
                             default_prompt_template=tourist_recommendation_prompt)

# Go ahead and ask a question:
prompt_node_gt5("What is the best city in Europe to live in?")


### Connecting an LLM to an in-memory document store

In [15]:
from haystack.document_stores import InMemoryDocumentStore
from datasets import load_dataset
from haystack.nodes import BM25Retriever, PromptTemplate, AnswerParser, PromptNode
import os
from haystack.pipelines import Pipeline
import openai
# Initialize Haystack's QA system
# Create a prompt node
openai_api_key = os.getenv("OPENAI_API_KEY")
document_store = InMemoryDocumentStore(use_bm25=True)


In [ ]:
dataset = load_dataset("cnn_dailymail", '3.0.0',  split="train")

In [16]:
# Step 1: Extract and Transform Data
transformed_documents = []

for i in range(500):
    content = dataset[i]['article']  
    transformed_documents.append({"content": content})

# Step 2: Write Transformed Data to Document Store
document_store.write_documents(transformed_documents)


Updating BM25 representation...: 100%|██████████| 482/482 [00:00<00:00, 2897.87 docs/s]


In [22]:

rag_prompt = PromptTemplate(
    prompt="""Synthesize a brief answer from the following text for the given question.
                             Provide a clear and concise response that summarizes the key points and information presented in the text.
                             Your answer should be in your own words and be no longer than 50 words.
                             \n\n Related text: {join(documents)} \n\n Question: {query} \n\n Answer:""",
    output_parser=AnswerParser(),
)

# Set up nodes
retriever = BM25Retriever(document_store=document_store, top_k=2)
pn = PromptNode("gpt-3.5-turbo", 
                api_key=openai_api_key, 
                model_kwargs={"stream":False},
                default_prompt_template=rag_prompt)

# Set up pipeline
pipe = Pipeline()
pipe.add_node(component=retriever, name="retriever", inputs=["Query"])
pipe.add_node(component=pn, name="prompt_node", inputs=["retriever"])

In [27]:
answer = pipe.run("What Pakistan's former prime minister was disqualified from participating in parliamentary elections")

In [31]:
answer.keys()

dict_keys(['answers', 'invocation_context', 'documents', 'root_node', 'params', 'query', 'node_id'])